In [121]:
!pip install --upgrade pip
#!pip install -U git+https://github.com/Techtonique/nnetsauce.git
#!pip install -U nnetsauce
!pip install -U ../.. --no-cache-dir 

# Both pip and git tried no difference

Python(55812) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(55841) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Processing /Users/t/Documents/Python_Packages/nnetsauce
  Preparing metadata (setup.py) ... done
  Created wheel for nnetsauce: filename=nnetsauce-0.30.4-py3-none-any.whl size=189590 sha256=56bb0f54e2b8348cc28881c8e3bb8ff2a03696cb2b6788bd881f6f24ff8b4bb9
  Stored in directory: /private/var/folders/cp/q8d6040n3m38d22z3hkk1zc40000gn/T/pip-ephem-wheel-cache-6u81kmb0/wheels/1a/03/81/1cc0cae33e498e24d4eec5887a4e621a55002440ceebce517d
Successfully built nnetsauce
  Attempting uninstall: nnetsauce
    Found existing installation: nnetsauce 0.30.4
    Uninstalling nnetsauce-0.30.4:
      Successfully uninstalled nnetsauce-0.30.4


### This code is copied from nnetasuce's MTS crross_val_score example:

In [122]:
import os 
import nnetsauce as ns 
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.datasets import load_diabetes
from sklearn.linear_model import ElasticNet, Ridge, LassoCV
from sklearn.model_selection import train_test_split
from statsmodels.tsa.base.datetools import dates_from_str
from nnetsauce.utils.model_selection import cross_val_score

#print(f"\n ----- Running: {os.path.basename(__file__)}... ----- \n")

# some example data
mdata = sm.datasets.macrodata.load_pandas().data
# prepare the dates index
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)
print(mdata.head())
mdata = mdata[['realgovt', 'tbilrate', 'cpi']]
mdata.index = pd.DatetimeIndex(quarterly)
data = np.log(mdata).diff().dropna()

n = data.shape[0]
max_idx_train = np.floor(n*0.9)
training_index = np.arange(0, max_idx_train)
testing_index = np.arange(max_idx_train, n)
df_train = data.iloc[training_index,:]
df_test = data.iloc[testing_index,:]

print(f"df_train.shape: {df_train.shape}")
regr = Ridge()
obj_MTS = ns.MTS(regr, lags = 3, 
                 n_hidden_features=7, 
                 seed=24, verbose = 0,
                 show_progress=False)

     year  quarter  realgdp  realcons  realinv  realgovt  realdpi   cpi  \
0 1959.00     1.00  2710.35   1707.40   286.90    470.05  1886.90 28.98   
1 1959.00     2.00  2778.80   1733.70   310.86    481.30  1919.70 29.15   
2 1959.00     3.00  2775.49   1751.80   289.23    491.26  1916.40 29.35   
3 1959.00     4.00  2785.20   1753.70   299.36    484.05  1931.30 29.37   
4 1960.00     1.00  2847.70   1770.50   331.72    462.20  1955.50 29.54   

      m1  tbilrate  unemp    pop  infl  realint  
0 139.70      2.82   5.80 177.15  0.00     0.00  
1 141.70      3.08   5.10 177.83  2.34     0.74  
2 140.50      3.82   5.30 178.66  2.74     1.09  
3 140.00      4.33   5.60 179.39  0.27     4.06  
4 139.60      3.50   5.20 180.01  2.31     1.19  
df_train.shape: (181, 3)


In [123]:
# Tried to set frequency for df_train but exception occurs
#df_train = df_train.asfreq("QE")

# TM: Not sure what 'QE' does, stick to the examples in the documentation (otherwise, you could contribute this to the package)

In [124]:
obj_MTS.fit(df_train)

MTS(lags=3, n_hidden_features=7, obj=Ridge(), seed=24, show_progress=False)

In [125]:
obj_MTS.input_dates
# Here we see the quarterly dates as in `quarterly` object

0      1959-06-30
1      1959-09-30
2      1959-12-31
3      1960-03-31
4      1960-06-30
          ...    
176    2003-06-30
177    2003-09-30
178    2003-12-31
179    2004-03-31
180    2004-06-30
Name: date, Length: 181, dtype: object

In [126]:
print(obj_MTS.predict(h=10))
# The output is right!

            realgovt  tbilrate  cpi
date                               
2004-09-30      0.00      0.04 0.01
2004-12-31      0.00     -0.05 0.01
2005-03-31      0.00      0.02 0.01
2005-06-30      0.00      0.03 0.01
2005-09-30      0.00      0.00 0.01
2005-12-31      0.00      0.01 0.01
2006-03-31      0.00      0.02 0.01
2006-06-30     -0.00      0.02 0.01
2006-09-30     -0.00      0.02 0.01
2006-12-31     -0.00      0.02 0.01


In [127]:
# obj_MTS.plot("realgovt") # if we want this to work, we need to ask for probabilistic forecasts at the beginning

In [128]:
obj_MTS.output_dates_
# The same

0    2004-09-30
1    2004-12-31
2    2005-03-31
3    2005-06-30
4    2005-09-30
5    2005-12-31
6    2006-03-31
7    2006-06-30
8    2006-09-30
9    2006-12-31
Name: date, dtype: object

In [129]:
#try:
print(obj_MTS.cross_val_score(df_train,                         
        n_jobs=None, 
        verbose = 1,
        initial_window=100,
        horizon=5,
        fixed_window=False,                         
        show_progress=True))

#except Exception as e:
#    print("\nException occurred:\n" + str(e))

#finally:
#    print("\n==> After cross_val_score the input dates are much longer:")
#    print("\n" + str(obj_MTS.input_dates))
#    print("\n==> I do predict and wee see daily data are not in line with original:")
#    print("\n" + str(obj_MTS.predict(h=10)))
#    print("\n" + str(obj_MTS.output_dates_))

  0%|          | 0/77 [00:00<?, ?it/s]

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
TEST: [100 101 102 103 104]


  1%|▏         | 1/77 [00:01<01:35,  1.25s/it]

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]
TEST: [101 102 103 104 105]


  3%|▎         | 2/77 [00:01<01:03,  1.18it/s]

...

 10%|█         | 8/77 [00:06<00:56,  1.22it/s]

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107]
TEST: [108 109 110 111 112]


 12%|█▏        | 9/77 [00:07<00:55,  1.22it/s]


TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108]
TEST: [109 110 111 112 113]


OutOfBoundsDatetime: Out of bounds timestamp: 2265-09-30 00:00:00 with frequency 'ns'

If I rerun the previous cell with `cross_val_score` then no exception occurs! Seemingly daily data, no overflow. 

### If I run `fit` several times the `input_data` attribute of `obj_MTS` always changes.

In [109]:
obj_MTS.fit(df_train)
obj_MTS.input_dates

0       1959-06-30
1       1959-07-01
2       1959-07-02
3       1959-07-03
4       1959-07-04
           ...    
1402    1963-05-02
1403    1963-05-03
1404    1963-05-04
1405    1963-05-05
1406    1963-05-06
Name: date, Length: 1407, dtype: object

In [110]:
obj_MTS.fit(df_train)
obj_MTS.input_dates

0       1959-06-30
1       1959-07-01
2       1959-07-02
3       1959-07-03
4       1959-07-04
           ...    
1583    1963-10-30
1584    1963-10-31
1585    1963-11-01
1586    1963-11-02
1587    1963-11-03
Name: date, Length: 1588, dtype: object